In [ ]:
#!pip install autotrain-advanced
#!pip install huggingface_hub

In [ ]:
#!autotrain setup --update-torch

In [ ]:
!pip install datasets transformers

In [4]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


In [ ]:
# Tải dataset từ Hugging Face (thay thế bằng dataset của bạn nếu cần)
dataset = load_dataset("csv", data_files="/content/Toxic_data_phase_2_official - Sheet2 (1).csv")

In [ ]:
# Load tokenizer và model
model_path = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
def preprocess_function(examples):
    """
    Hàm xử lý dữ liệu trước khi huấn luyện.
    """
    inputs = examples["toxic_text"]  # Lấy nội dung từ cột "toxic_text"
    targets = examples["paraphrased_text"]  # Lấy nội dung từ cột "paraphrase_text"

    # Tokenize inputs and targets using the tokenizer
    model_inputs = tokenizer(
        inputs, max_length=256, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():  # Use as_target_tokenizer for labels
        labels = tokenizer(
            targets, max_length=256, truncation=True, padding="max_length"
        )

    # Assign labels to the model_inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Tiền xử lý dữ liệu
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
#df = tokenized_datasets.select(range(5)).to_pandas()
##Display the DataFrame
#df.head()

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
if torch.cuda.is_available():
    print("GPU đang được sử dụng")
else:
    print("GPU không được sử dụng")

GPU đang được sử dụng


In [ ]:
# Cấu hình tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",          # Thư mục lưu model
    evaluation_strategy="steps",
    eval_steps=50,                  # Evaluate every 500 steps
    learning_rate=5e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True if torch.cuda.is_available() else False,
    gradient_accumulation_steps=8,  # Added gradient accumulation
    gradient_checkpointing=True,  # Enable gradient checkpointing
    report_to="wandb",                # Enable W&B logging
    run_name="thử nghiệm",  # Optional: Give your run a name
)


In [ ]:
# ipython-input-31-b176e2889ee9
# Định nghĩa hàm đánh giá
def compute_metrics(pred):
    """
    Hàm đánh giá mô hình, tính toán các metrics theo từng batch.
    """
    labels = pred.label_ids
    preds = pred.predictions
    preds = np.argmax(preds, axis=-1)

    # Khởi tạo các biến lưu trữ metrics
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    total_accuracy = 0
    total_samples = 0

    # Chia dữ liệu thành các batch
    batch_size = 32  # Kích thước batch
    num_batches = len(labels) // batch_size + (len(labels) % batch_size != 0)

    # Tính toán metrics cho từng batch
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(labels))

        batch_labels = labels[start_idx:end_idx].flatten()
        batch_preds = preds[start_idx:end_idx].flatten()

        # Tính toán metrics cho batch hiện tại
        precision, recall, f1, _ = precision_recall_fscore_support(batch_labels, batch_preds, average='weighted')
        accuracy = accuracy_score(batch_labels, batch_preds)

        # Cập nhật tổng metrics
        total_precision += precision * len(batch_labels)
        total_recall += recall * len(batch_labels)
        total_f1 += f1 * len(batch_labels)
        total_accuracy += accuracy * len(batch_labels)
        total_samples += len(batch_labels)

    # Tính toán metrics trung bình
    avg_precision = total_precision / total_samples
    avg_recall = total_recall / total_samples
    avg_f1 = total_f1 / total_samples
    avg_accuracy = total_accuracy / total_samples

    metrics = {"precision": avg_precision, "recall": avg_recall, "f1": avg_f1, "accuracy": avg_accuracy}

    # Hiển thị kết quả đánh giá dưới dạng bảng
    df_metrics = pd.DataFrame([metrics])
    print("\nKết quả đánh giá mô hình:\n", df_metrics)

    return metrics

In [ ]:
# Trainer để huấn luyện mô hình
# Apply train_test_split to the tokenized dataset
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)
# Access the splits using the keys created by train_test_split
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Use the train split from train_test_split
    eval_dataset=eval_dataset,  # Use the test split from train_test_split
    compute_metrics=compute_metrics
)

In [ ]:
import wandb
wandb.login()


In [ ]:
# Huấn luyện
trainer.train()

In [ ]:
# Lưu model sau khi train
trainer.save_model("./ViT5-base-finetuned")
tokenizer.save_pretrained("./ViT5-base-finetuned")


In [ ]:
!pip install huggingface_hub
!pip install --upgrade huggingface_hub


In [ ]:
from huggingface_hub import notebook_login, login, HfApi

notebook_login()

In [ ]:
from huggingface_hub import HfApi, create_repo, upload_folder


In [ ]:

# Tên repo trên Hugging Face (Thay thế bằng tên của bạn)
repo_name = "Nosiath/ViT5-base-finetuned"

# Tạo repository trên Hugging Face (nếu chưa có)
create_repo(repo_name,exist_ok=True)

# Đẩy toàn bộ mô hình lên Hugging Face Hub
upload_folder(
    folder_path="./ViT5-base-finetuned",
    repo_id=repo_name,
    commit_message="Upload trained model"
)

print(f"🚀 Mô hình đã được đẩy lên: https://huggingface.co/{repo_name}")

In [6]:
model_path = "Nosiath/ViT5-base-finetuned"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [10]:
def convert_toxic_to_polite(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=200, truncation=True)
    inputs = inputs.to(device)
    model.to(device)

    ## Chuyển đổi input_ids sang token
    #input_ids = inputs['input_ids'].cpu().numpy()[0]
    #tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #print("Sau khi tách từ ra: ")
    #print(tokens)

    outputs = model.generate(
        **inputs,
        max_length=200,
        no_repeat_ngram_size=3,
        num_beams=25,
        early_stopping=True,
        length_penalty=30.0,
        min_length=5,
        repetition_penalty=1.5
    )
    polite_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return polite_sentence



In [11]:
#Kiểm tra kết quả
test_sentences = [
    "Cái thằng quỷ này, sao mà mày ngu lol quá vậy hả, óc chó vừa vừa thôi, tại sao không suy nghĩ trước khi nói vậy?"
]

print("Kết quả chuyển đổi câu:")
for sentence in test_sentences:
    polite_sentence = convert_toxic_to_polite(sentence)
    print(f"Thô tục: {sentence}")
    print(f"Lịch sự: {polite_sentence}\n")

Kết quả chuyển đổi câu:
Thô tục: Cái thằng quỷ này, sao mà mày ngu lol quá vậy hả, óc chó vừa vừa thôi, tại sao không suy nghĩ trước khi nói vậy?
Lịch sự: Tại sao bạn lại có suy nghĩ kỳ cục như vậy nhỉ, đầu óc bạn thật quá kém đấy, tại sao bạn không suy nghĩ trước khi nói vậy?

